In [32]:
import pandas as pd
import numpy as np
import torch

In [33]:
class ToSoccerMapTensor:
    """Convert inputs to a spatial representation.

    Parameters
    ----------
    dim : tuple(int), default=(68, 104)
        The dimensions of the pitch in the spatial representation.
    """

    def __init__(self, dim=(68, 104)):
        assert len(dim) == 2
        self.y_bins, self.x_bins = dim

    def _get_cell_indexes(self, x, y):
        x_bin = np.clip((x + 52.5) / 105 * self.x_bins, 0, self.x_bins - 1).astype(np.uint8)
        y_bin = np.clip((y + 34) / 68 * self.y_bins, 0, self.y_bins - 1).astype(np.uint8)
        return x_bin, y_bin

    def __call__(self, sample):
        start_x, start_y, end_x, end_y = (
            sample["ball_x_start"],
            sample["ball_y_start"],
            sample["ball_x_end"],
            sample["ball_y_end"],
        )
        speed_x, speed_y = sample["vx_player_201"], sample["vy_player_201"]
        frame = sample["frame"]

        pass_outcome_type = sample.get("pass_outcome_type")
        pass_outcome_type_mapped = pass_outcome_mapping.get(pass_outcome_type, None)
        if pass_outcome_type_mapped is None:
            raise ValueError(f"Invalid pass_outcome_type: {pass_outcome_type}")
        target = int(pass_outcome_type_mapped)

        ball_coo = np.array([[start_x, start_y]])
        goal_coo = np.array([[52.5, 0]])

        players_att_coo = frame.loc[frame["team_id"] == sample["team_id"], ["x_player_201", "y_player_201"]].values.reshape(-1, 2)
        players_def_coo = frame.loc[frame["team_id"] != sample["team_id"], ["x_player_201", "y_player_201"]].values.reshape(-1, 2)

        matrix = np.zeros((15, self.y_bins, self.x_bins))

        # Channel 1: Locations of attacking team
        x_bin_att, y_bin_att = self._get_cell_indexes(players_att_coo[:, 0], players_att_coo[:, 1])
        matrix[0, y_bin_att, x_bin_att] = 1

        # Channel 2: Locations of defending team
        x_bin_def, y_bin_def = self._get_cell_indexes(players_def_coo[:, 0], players_def_coo[:, 1])
        matrix[1, y_bin_def, x_bin_def] = 1

        # Channel 3: Distance to ball
        yy, xx = np.ogrid[0.5: self.y_bins, 0.5: self.x_bins]
        x0_ball, y0_ball = self._get_cell_indexes(ball_coo[:, 0], ball_coo[:, 1])
        matrix[2, :, :] = np.sqrt((xx - x0_ball) * 2 + (yy - y0_ball) * 2)

        # Channel 4: Distance to goal
        x0_goal, y0_goal = self._get_cell_indexes(goal_coo[:, 0], goal_coo[:, 1])
        matrix[3, :, :] = np.sqrt((xx - x0_goal) * 2 + (yy - y0_goal) * 2)

        # Channel 5: Cosine of the angle between the ball and goal
        coords = np.dstack(np.meshgrid(xx, yy))
        goal_coo_bin = np.concatenate((x0_goal, y0_goal))
        ball_coo_bin = np.concatenate((x0_ball, y0_ball))
        a = goal_coo_bin - coords
        b = ball_coo_bin - coords
        matrix[4, :, :] = np.clip(np.sum(a * b, axis=2) / (np.linalg.norm(a, axis=2) * np.linalg.norm(b, axis=2)), -1, 1)

        # Channel 6: Sine of the angle between the ball and goal
        matrix[5, :, :] = np.sqrt(1 - matrix[4, :, :] ** 2)

        # Channel 7: Angle to the goal location
        matrix[6, :, :] = np.abs(np.arctan2((y0_goal - coords[:, :, 1]), (x0_goal - coords[:, :, 0])))

        # Channels 8-9: Ball speed
        matrix[7, y0_ball, x0_ball] = speed_x
        matrix[8, y0_ball, x0_ball] = speed_y

        # Channel 10: Number of possession team’s players between the ball and every other location
        dist_att_goal = matrix[0, :, :] * matrix[3, :, :]
        dist_att_goal[dist_att_goal == 0] = np.nan
        dist_ball_goal = matrix[3, y0_ball, x0_ball]
        player_in_front_of_ball = dist_att_goal <= dist_ball_goal

        outplayed1 = lambda x: np.sum(player_in_front_of_ball & (x <= dist_ball_goal) & (dist_att_goal >= x))
        matrix[9, :, :] = np.vectorize(outplayed1)(matrix[3, :, :])

        # Channel 11: Number of opponent team’s players between the ball and every other location
        dist_def_goal = matrix[1, :, :] * matrix[3, :, :]
        dist_def_goal[dist_def_goal == 0] = np.nan
        dist_ball_goal = matrix[3, y0_ball, x0_ball]
        player_in_front_of_ball = dist_def_goal <= dist_ball_goal

        outplayed2 = lambda x: np.sum(player_in_front_of_ball & (x <= dist_ball_goal) & (dist_def_goal >= x))
        matrix[10, :, :] = np.vectorize(outplayed2)(matrix[3, :, :])

        # Channel 12: Carrier's velocity
        carrier_velocity = sample["carrier_velocity"]
        matrix[11, y0_ball, x0_ball] = carrier_velocity

        # Channels 13-14: Possession team players' velocity (x and y)
        players_vel_coo = frame.loc[frame["team_id"] == sample["team_id"], ["vx_player_201", "vy_player_201"]].values
        x_bin_att_vel, y_bin_att_vel = self._get_cell_indexes(players_att_coo[:, 0], players_att_coo[:, 1])
        matrix[12, y_bin_att_vel, x_bin_att_vel] = players_vel_coo[:, 0]
        matrix[13, y_bin_att_vel, x_bin_att_vel] = players_vel_coo[:, 1]

        # Channel 15: Distance to event's origin location
        x0_start, y0_start = self._get_cell_indexes(np.array([start_x]), np.array([start_y]))
        matrix[14, :, :] = np.sqrt((xx - x0_start) * 2 + (yy - y0_start) * 2)

        mask = np.zeros((1, self.y_bins, self.x_bins))
        end_ball_coo = np.array([[end_x, end_y]])
        if np.isnan(end_ball_coo).any():
            raise ValueError("End coordinates not known.")
        x0_ball_end, y0_ball_end = self._get_cell_indexes(end_ball_coo[:, 0], end_ball_coo[:, 1])
        mask[0, y0_ball_end, x0_ball_end] = 1

        return (
            torch.from_numpy(matrix).float(),
            torch.from_numpy(mask).float(),
            torch.tensor([target]).float()
        )

In [34]:
df = pd.read_csv('final_pass_track_4440.csv')

In [35]:
pass_outcome_mapping = {
    'C': 0,
    'D': 1,
    'B': 2,
    'O': 3,
    'S': 4
}
sample = {
    "ball_x_start": df.iloc[1:2]["ball_x_start"].iloc[0],
    "ball_y_start": df.iloc[1:2]["ball_y_start"].iloc[0],
    "ball_x_end": df.iloc[1:2]["ball_x_end"].iloc[0],
    "ball_y_end": df.iloc[1:2]["ball_y_end"].iloc[0],
    "vx_player_201": df.iloc[1:2]["vx_player_201"].iloc[0],
    "vy_player_201": df.iloc[1:2]["vy_player_201"].iloc[0],
    "pass_outcome_type": df.iloc[1:2]["pass_outcome_type"].iloc[0],
    "team_id": df.iloc[1:2]["team_id"].iloc[0],
    "carrier_velocity": df.iloc[1:2]["carrier_velocity"].iloc[0],
    "frame": df.iloc[1:2]  
}

tensor_converter = ToSoccerMapTensor()
matrix, mask, target = tensor_converter(sample)

/tmp/ipykernel_213826/177791138.py:54: RuntimeWarning: invalid value encountered in sqrt
  matrix[2, :, :] = np.sqrt((xx - x0_ball) * 2 + (yy - y0_ball) * 2)
/tmp/ipykernel_213826/177791138.py:58: RuntimeWarning: invalid value encountered in sqrt
  matrix[3, :, :] = np.sqrt((xx - x0_goal) * 2 + (yy - y0_goal) * 2)
/tmp/ipykernel_213826/177791138.py:108: RuntimeWarning: invalid value encountered in sqrt
  matrix[14, :, :] = np.sqrt((xx - x0_start) * 2 + (yy - y0_start) * 2)


In [23]:
has_nan = torch.isnan(matrix)

nan_indices = [i for i in range(matrix.size(0)) if torch.any(has_nan[i])]

print("Índices com NaNs na primeira dimensão:", nan_indices)

Índices com NaNs na primeira dimensão: [2, 3, 14]


In [31]:
int(target[0])

0

In [58]:
df.iloc[:1]

,game_id,period,game_event_id,possession_event_id,player_id,shirt_number,possession_event_type,position_group_type,team_id,home_team,...,angle_to_goal_left,angle_to_goal_right,angle_to_goal,sin_angle_possession,cos_angle_possession,opponent_team_velocity_x,opponent_team_velocity_y,carrier_velocity,other_players_location,possession_team_velocity
0,4440,1.0,4466727.0,4283224.0,5076.0,17.0,PA,CF,119.0,False,...,3.115497,0.026543,0.026543,0.02654,0.999648,NaN,NaN,0.577062,"[(-39.075, -0.667)]",NaN


In [48]:
(spatial_representations.size)

1

In [50]:
type(spatial_representations)

pandas.core.series.Series

In [49]:
(len(spatial_representations[0]))

3

In [51]:
type(spatial_representations[0])

tuple

In [52]:
(len(spatial_representations[0][0]))

19

In [53]:
type(spatial_representations[0][0])

torch.Tensor

In [55]:
(len(spatial_representations[0][0][0]))

68

In [54]:
type(spatial_representations[0][0][0])

torch.Tensor

In [56]:
(len(spatial_representations[0][0][0][0]))

104

In [57]:
type(spatial_representations[0][0][0][0])

torch.Tensor

(pandas.core.series.Series,tuple,torch.Tensor,torch.Tensor)
(1,3,19,68,104)

In [78]:
class ToSoccerMapTensor:
    """Convert inputs to a spatial representation.

    Parameters
    ----------
    dim : tuple(int), default=(68, 104)
        The dimensions of the pitch in the spatial representation.
    """

    def __init__(self, dim=(68, 104)):
        assert len(dim) == 2
        self.y_bins, self.x_bins = dim

    def _get_cell_indexes(self, x, y):
        x_bin = np.clip((x + 52.5) / 105 * self.x_bins, 0, self.x_bins - 1).astype(np.uint8)
        y_bin = np.clip((y + 34) / 68 * self.y_bins, 0, self.y_bins - 1).astype(np.uint8)
        return x_bin, y_bin

    def __call__(self, sample):
        start_x, start_y, end_x, end_y = (
            sample["ball_x_start"],
            sample["ball_y_start"],
            sample["ball_x_end"],
            sample["ball_y_end"],
        )
        speed_x, speed_y = sample["vx_player_201"], sample["vy_player_201"]
        frame = sample["frame"]

        pass_outcome_type = sample.get("pass_outcome_type")
        pass_outcome_type_mapped = pass_outcome_mapping.get(pass_outcome_type, None)
        if pass_outcome_type_mapped is None:
            raise ValueError(f"Invalid pass_outcome_type: {pass_outcome_type}")
        target = int(pass_outcome_type_mapped)

        ball_coo = np.array([[start_x, start_y]])
        goal_coo = np.array([[52.5, 0]])

        players_att_coo = frame.loc[frame["team_id"] == sample["team_id"], ["x_player_201", "y_player_201"]].values.reshape(-1, 2)
        players_def_coo = frame.loc[frame["team_id"] != sample["team_id"], ["x_player_201", "y_player_201"]].values.reshape(-1, 2)

        matrix = np.zeros((15, self.y_bins, self.x_bins))

        # Channel 1: Locations of attacking team
        x_bin_att, y_bin_att = self._get_cell_indexes(players_att_coo[:, 0], players_att_coo[:, 1])
        matrix[0, y_bin_att, x_bin_att] = 1

        # Channel 2: Locations of defending team
        x_bin_def, y_bin_def = self._get_cell_indexes(players_def_coo[:, 0], players_def_coo[:, 1])
        matrix[1, y_bin_def, x_bin_def] = 1

        # Channel 3: Distance to ball
        yy, xx = np.ogrid[0.5: self.y_bins, 0.5: self.x_bins]
        x0_ball, y0_ball = self._get_cell_indexes(ball_coo[:, 0], ball_coo[:, 1])
        matrix[2, :, :] = np.sqrt((xx - x0_ball) * 2 + (yy - y0_ball) * 2)

        # Channel 4: Distance to goal
        x0_goal, y0_goal = self._get_cell_indexes(goal_coo[:, 0], goal_coo[:, 1])
        matrix[3, :, :] = np.sqrt((xx - x0_goal) * 2 + (yy - y0_goal) * 2)

        # Channel 5: Cosine of the angle between the ball and goal
        coords = np.dstack(np.meshgrid(xx, yy))
        goal_coo_bin = np.concatenate((x0_goal, y0_goal))
        ball_coo_bin = np.concatenate((x0_ball, y0_ball))
        a = goal_coo_bin - coords
        b = ball_coo_bin - coords
        matrix[4, :, :] = np.clip(np.sum(a * b, axis=2) / (np.linalg.norm(a, axis=2) * np.linalg.norm(b, axis=2)), -1, 1)

        # Channel 6: Sine of the angle between the ball and goal
        matrix[5, :, :] = np.sqrt(1 - matrix[4, :, :] ** 2)

        # Channel 7: Angle to the goal location
        matrix[6, :, :] = np.abs(np.arctan2((y0_goal - coords[:, :, 1]), (x0_goal - coords[:, :, 0])))

        # Channels 8-9: Ball speed
        matrix[7, y0_ball, x0_ball] = speed_x
        matrix[8, y0_ball, x0_ball] = speed_y

        # Channel 10: Number of possession team’s players between the ball and every other location
        dist_att_goal = matrix[0, :, :] * matrix[3, :, :]
        dist_att_goal[dist_att_goal == 0] = np.nan
        dist_ball_goal = matrix[3, y0_ball, x0_ball]
        player_in_front_of_ball = dist_att_goal <= dist_ball_goal

        outplayed1 = lambda x: np.sum(player_in_front_of_ball & (x <= dist_ball_goal) & (dist_att_goal >= x))
        matrix[9, :, :] = np.vectorize(outplayed1)(matrix[3, :, :])

        # Channel 11: Number of opponent team’s players between the ball and every other location
        dist_def_goal = matrix[1, :, :] * matrix[3, :, :]
        dist_def_goal[dist_def_goal == 0] = np.nan
        dist_ball_goal = matrix[3, y0_ball, x0_ball]
        player_in_front_of_ball = dist_def_goal <= dist_ball_goal

        outplayed2 = lambda x: np.sum(player_in_front_of_ball & (x <= dist_ball_goal) & (dist_def_goal >= x))
        matrix[10, :, :] = np.vectorize(outplayed2)(matrix[3, :, :])

        # Channel 12: Carrier's velocity
        carrier_velocity = sample["carrier_velocity"]
        matrix[11, y0_ball, x0_ball] = carrier_velocity

        # Channels 13-14: Possession team players' velocity (x and y)
        players_vel_coo = frame.loc[frame["team_id"] == sample["team_id"], ["vx_player_201", "vy_player_201"]].values
        x_bin_att_vel, y_bin_att_vel = self._get_cell_indexes(players_att_coo[:, 0], players_att_coo[:, 1])
        matrix[12, y_bin_att_vel, x_bin_att_vel] = players_vel_coo[:, 0]
        matrix[13, y_bin_att_vel, x_bin_att_vel] = players_vel_coo[:, 1]

        # Channel 15: Distance to event's origin location
        x0_start, y0_start = self._get_cell_indexes(np.array([start_x]), np.array([start_y]))
        matrix[14, :, :] = np.sqrt((xx - x0_start) * 2 + (yy - y0_start) * 2)

        mask = np.zeros((1, self.y_bins, self.x_bins))
        end_ball_coo = np.array([[end_x, end_y]])
        if np.isnan(end_ball_coo).any():
            raise ValueError("End coordinates not known.")
        x0_ball_end, y0_ball_end = self._get_cell_indexes(end_ball_coo[:, 0], end_ball_coo[:, 1])
        mask[0, y0_ball_end, x0_ball_end] = 1

        return (
            torch.from_numpy(matrix).float(),
            torch.from_numpy(mask).float(),
            torch.tensor([target]).float()
        )
pass_outcome_mapping = {
    'C': 0,
    'D': 1,
    'B': 2,
    'O': 3,
    'S': 4
}
sample = {
    "ball_x_start": df.iloc[1:2]["ball_x_start"].iloc[0],
    "ball_y_start": df.iloc[1:2]["ball_y_start"].iloc[0],
    "ball_x_end": df.iloc[1:2]["ball_x_end"].iloc[0],
    "ball_y_end": df.iloc[1:2]["ball_y_end"].iloc[0],
    "vx_player_201": df.iloc[1:2]["vx_player_201"].iloc[0],
    "vy_player_201": df.iloc[1:2]["vy_player_201"].iloc[0],
    "pass_outcome_type": df.iloc[1:2]["pass_outcome_type"].iloc[0],
    "team_id": df.iloc[1:2]["team_id"].iloc[0],
    "carrier_velocity": df.iloc[1:2]["carrier_velocity"].iloc[0],
    "frame": df.iloc[1:2]  
}

tensor_converter = ToSoccerMapTensor()
matrix, mask, target = tensor_converter(sample)

/tmp/ipykernel_11417/2553297429.py:54: RuntimeWarning: invalid value encountered in sqrt
  matrix[2, :, :] = np.sqrt((xx - x0_ball) * 2 + (yy - y0_ball) * 2)
/tmp/ipykernel_11417/2553297429.py:58: RuntimeWarning: invalid value encountered in sqrt
  matrix[3, :, :] = np.sqrt((xx - x0_goal) * 2 + (yy - y0_goal) * 2)
/tmp/ipykernel_11417/2553297429.py:108: RuntimeWarning: invalid value encountered in sqrt
  matrix[14, :, :] = np.sqrt((xx - x0_start) * 2 + (yy - y0_start) * 2)


In [81]:
target

tensor([0.])